In [ ]:
from one.api import ONE
import numpy as np
import pyvista as pv
import vvasp
from iblatlas.regions import BrainRegions
from iblatlas.atlas import AllenAtlas, Insertion
from labdata.schema import *
from labdata import chronic_paper as cp

one = ONE()
br = BrainRegions()
ba = AllenAtlas()

In [ ]:
#pids = one.search_insertions(atlas_acronym=['CP'])
subjects = np.unique(cp.ChronicInsertion().fetch('subject_name', order_by='procedure_datetime')).tolist()
#Subject()
subjects

In [ ]:
ProbeInsertion() * Probe() & dict(subject_name='MM013')
(ProbeInsertion() * Probe() & 'probe_n_shanks=1')

In [ ]:
plotter = pv.Plotter()
atlas = vvasp.VVASPAtlas(plotter)

sub = 'MM013'

insertions = (ProbeInsertion() * Probe() & dict(subject_name=sub)).fetch(as_dict=True)
ikey = (ProbeInsertion() * Probe() & dict(subject_name=sub)).fetch('KEY', as_dict=True)
print(f'\n\nThere are {len(insertions)} insertions for {sub}')
prbs = []
for ins in insertions:
#for ins in [insertions[0]]:
    angles = np.array([ins['insertion_el'], ins['insertion_spin'], ins['insertion_az']])
    entry = np.array([ins['insertion_ml'], ins['insertion_ap']])
    depth = ins['insertion_depth']
    # TODO: get probetype first 
    if ins['probe_n_shanks'] == 1:
        prb = vvasp.Probe('NP1', plotter)
    elif ins['probe_n_shanks'] == 4:
        prb = vvasp.Probe('NP24', plotter)
    else:
        raise ValueError('Probe type not supported')
    prb.drive_probe_from_entry(entry, angles, ins['insertion_depth'], root_mesh=atlas.meshes['root']) # position the probe via an entry point, angles and driven depth
    _, region_acronyms = prb.compute_region_intersections(atlas)
    prbs.append(prb)
    region_acronyms = np.unique([item for sublist in region_acronyms for item in sublist])
    region_acronyms = np.unique(br.acronym2acronym(region_acronyms, mapping='Beryl'))
    region_acronyms = [x for x in region_acronyms if x != 'Outside atlas']
    region_acronyms = [x for x in region_acronyms if x[0].isupper()]
    _ = [atlas.add_atlas_region_mesh(r) for r in region_acronyms]
    print(region_acronyms)
    pids = [one.search_insertions(atlas_acronym=ra, project='brainwide') for ra in region_acronyms]
    pids = np.unique(np.concatenate(pids, axis=0))
plotter.show()

In [ ]:
pid = '57656bee-e32e-4848-b924-0f6f18cfdfb1'
traj = one.alyx.rest('trajectories', 'list', probe_insertion=pid,
                         provenance='Ephys aligned histology track')
assert len(traj) == 1
traj = traj[0]
ins = Insertion.from_dict(traj, brain_atlas=ba)
prb = vvasp.Probe('NP1', plotter)
tip = ins.tip * 1e6
entry = np.array([traj['x'], traj['y']])
angles = np.array([90 + traj['theta'], traj['roll'], traj['phi'] - 90 + 180])
angles = np.array([90 - traj['theta'], traj['roll'], traj['phi'] - 90])
#angles = np.array([traj['theta'] - 90, traj['roll'], traj['phi']])
prb.set_location(tip, angles)
#prb.drive_probe_from_entry(entry, angles, traj['depth'], root_mesh=atlas.meshes['root']) # position the probe via an entry point, angles and driven depth
prb.make_inactive()

_, region_acronyms = prb.compute_region_intersections(atlas)

In [ ]:
entry = dict(**ikey[2],
             pid ='57acb640-4f64-4a89-bedf-69351a3aea75',)
#cp.IBLMatchedInsertion().insert1(entry)

In [ ]:
cp.IBLMatchedInsertion()

In [ ]:
matched_insertions = cp.IBLMatchedInsertion().fetch(as_dict=True)
for m in matched_insertions:
    pid = m.pop('pid')
    eid, probename = one.pid2eid(pid)
    session_dict = one.eid2ref(eid)

    # 1. insert the subject if it's not there
    alyx_sub = one.alyx.rest('subjects', 'read', id=session_dict['subject'])
    subj_dict = dict(subject_name=f'_{session_dict["subject"]}',
                     subject_dob=alyx_sub['birth_date'],
                     subject_sex=alyx_sub['sex'],
                     user_name='mmelin',
                     strain_name='C57BL/6J')
    print(subj_dict)
    Subject().insert1(subj_dict, skip_duplicates=True)

    # 2. Download the session data
    bin_dset = one.list_datasets(eid, '*.ap.cbin', collection=f'raw_ephys_data/{probename}')
    meta_dset = one.list_datasets(eid, '*.ap.meta', collection=f'raw_ephys_data/{probename}')
    assert len(bin_dset) == 1
    assert len(meta_dset) == 1
    bin_dset = bin_dset[0]
    meta_dset = meta_dset[0]

    bin_path = one.load_dataset(eid, bin_dset, download_only=True)
    meta_path = one.load_dataset(eid, meta_dset, download_only=True)

    # 3. insert the ephys session

    
    # 4. clean up (delete the local files that were downloaded)

In [ ]:
bin_path

In [ ]:
Subject()

In [ ]:
json = one.alyx.rest('subjects', 'read', id='MM012')
json

In [ ]:
dic